In [1]:
%cd Documents/aiClub/lltm/LLTM/backend

/home/grall/Documents/aiClub/lltm/LLTM/backend


In [2]:
from dictionary import get_word_info

/usr/local/lib/python3.8/dist-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


In [5]:
get_word_info('vauva', 'Finnish', True)

{'ipa': '/ˈʋɑu̯ʋɑ/',
 'img': None,
 'definitions': {'Noun': {('baby', 'a very young child'): [],
   ('informal', '(informal) baby animal'): []}}}

In [9]:
import requests
from bs4 import BeautifulSoup

In [10]:
def parse_def_list(ol, raw_text=False):
    decode = (lambda x: x.text) if raw_text else (lambda x: x.decode_contents())

    result = {}
    for li in ol.find_all('li', recursive=False):
        sub_ol = li.select_one('ol')

        a_definition = decode(li.select_one('a'))
        description = li.select(':scope > *:not(ol):not(dl):not(style)')
        description = [decode(x) for x in description]

        if len(description) == 4 and description[-1] == ')' and description[-3] == '(':
            description = description[-2]
        else:
            description = ' '.join(description)

        if sub_ol:
            result[(a_definition, description)] = parse_def_list(sub_ol, raw_text)
        else:
            examples = [decode(ex) for ex in li.select('.h-usage-example')]
            result[(a_definition, description)] = examples
    return result

In [11]:
# word = 'hyvä'
word = 'koira'
# word = 'talo'

html = requests.get(f'https://en.wiktionary.org/api/rest_v1/page/html/{word}').text
soup = BeautifulSoup(html)

WORD_TYPES = ['Adjective', 'Noun', 'Verb', 'Adverb', 'Article']

fn = soup.select_one('section:has(> h2:-soup-contains("Finnish"))')

pronunciation = fn.select_one('section:has(> h3:-soup-contains("Pronunciation"))')
word_type_data = {
    t: fn.select_one(f'section:has(> h3:-soup-contains("{t}"))')
    for t in WORD_TYPES
}

section_titles = [x.text for x in fn.select('h3', recursive=False)]
for title in WORD_TYPES + ['Etymology', 'Pronunciation', 'References', 'Further reading', 'Anagrams', 'See also']:
    if title in section_titles:
        section_titles.remove(title)

if section_titles:
    print('UNHANDLED SECTION TITLES:', section_titles)

definitions = {
    t: parse_def_list(d.select_one('ol'), True)
    for t, d in word_type_data.items()
    if d
}

ipa = next(ipa.text for ipa in pronunciation.select('.IPA') if ipa.text[0]=='/')

ipa

'/ˈkoi̯rɑ/'

In [24]:
'https:' + fn.select_one('figure[typeof="mw:File/Thumb"] img')['src']

'https://upload.wikimedia.org/wikipedia/commons/thumb/e/e7/Jymy_and_Senni_are_taking_a_rest_%2816039954181%29.jpg/220px-Jymy_and_Senni_are_taking_a_rest_%2816039954181%29.jpg'

In [133]:
definitions

{'Noun': {('dog',
   'dog ( Canis lupus familiaris ) 1548, Mikael Agricola, transl., Se Wsi Testamenti [The New Testament], Stockholm, pyhen Paualin Epistola Philipperin tyge (Philippians) 3:2:Cauattacat teitenne Coirilda[Kavahtakaat teitänne koirilta]Beware of dogs'): [],
  ('Ellipsis',
   'Ellipsis of koiranuinti (“dog paddle (swimming stroke)”) . '): [],
  ('military', '(military slang) military police'): []}}

In [92]:
ol = adj.select_one('ol')


In [94]:
def rec_print(data, indent=0):
    for (a_def, desc), v in data.items():
        print(' '*indent, end='')
        print(a_def, '|', desc)
        if type(v) is dict:
            rec_print(v, indent=indent+4)
        if type(v) is list:
            for ex in v:
                print(' '*indent, end='')
                print('  -', ex)

# rec_print(
parse_def_list(
    adj.select_one('ol')
)
# )

{('good',
  'good'): {('good',
   '<span class="Latn" lang="en">useful for a particular purpose</span>'): {('good',
    '<span class="Latn" lang="en">of high quality</span>'): [],
   ('good',
    'good ( <span class="Latn" lang="en">competent, skilled, talented</span> ) [ with   <a class="extiw" href="https://en.wikipedia.org/wiki/inessive%20case" rel="mw:WikiLink/Interwiki" title="w:inessive case">inessive</a>   or   ( with verbs )   illative of <a href="./Appendix:Finnish_verb_forms#Infinitives" rel="mw:WikiLink" title="Appendix:Finnish verb forms">third infinitive</a>   ‘at’ ]'): ['<i class="Latn mention e-example" lang="fi"><b>hyvä</b> uinnissa <span typeof="mw:Entity">/</span> <b>hyvä</b> uimaan</i> ― <span class="e-translation"><b>good</b> at swimming</span>'],
   ('good',
    '<span class="Latn" lang="en">effective, that has the desired effect</span>'): [],
   ('good',
    '<span class="Latn" lang="en">beneficial<span typeof="mw:Entity">;</span> worthwhile, helpful</span>'): [],

In [80]:
# def extract_definition_list(ol):
#     definitions = []
#     for li in ol.find_all('li', recursive=False):
#         sub_ol = li.find('ol')
#         if sub_ol:
#             definitions.append(extract_definition_list(sub_ol))
            
#         translation = li.text
#         examples = [ex.text for ex in li.select('.h-usage-example')]
#         definition = (translation, examples)
#         # print(definition)
#         definitions.append(definition)
#     return definitions

def extract_definition_list(ol):
    result = {}
    for li in ol.find_all('li', recursive=False):
        sub_ol = li.find('ol')
        if sub_ol:
            result[li.text] = extract_definition_list(sub_ol)
        else:
            translation = li.text
            examples = [ex.text for ex in li.select('.h-usage-example')]
            result[li.text] = (translation, examples)
    return result

In [74]:
import json

In [81]:
print(
json.dumps(
    extract_definition_list(adj.find_all('ol', recursive=False)[0]),
    indent=4
)
)

{
    "good\nSynonyms: see Thesaurus:hyv\u00e4\ngood (useful for a particular purpose)\nAntonyms: huono, kehno\ngood (of high quality)\ngood (competent, skilled, talented) [with inessive or (with verbs) illative of third infinitive \u2018at\u2019]\nhyv\u00e4 uinnissa / hyv\u00e4 uimaan \u2015 good at swimming\ngood (effective, that has the desired effect)\ngood (beneficial; worthwhile, helpful)\ngood (of food, edible; not stale or rotten)\ngood (acting in the interest of good; ethical good intentions)\nAntonym: paha\nKaikilla on taito tehd\u00e4 hyv\u00e4\u00e4. \u2015 Everyone has the ability to do good (deeds).\ngood (healthful, healthy, promoting health) (usually in the translative singular)\nAntonym: paha\nolla hyv\u00e4ksi jollekulle \u2015 to be good/healthy for someone\nL\u00e4mmin tee tekee hyv\u00e4\u00e4 kurkkukivulle. \u2015 Warm tea is good for a sore throat.\ngood (pleasant, agreeable)\nAntonym: huono\nMinun tekee hyv\u00e4\u00e4 k\u00e4yd\u00e4 v\u00e4lill\u00e4 lenkill\u

In [43]:
for ol in adj.find_all('ol'):
    extract_definition_list(ol)

('good (of high quality)', [])
('good (competent, skilled, talented) [with inessive or (with verbs) illative of third infinitive ‘at’]\nhyvä uinnissa / hyvä uimaan ― good at swimming', ['hyvä uinnissa / hyvä uimaan ― good at swimming'])
('good (effective, that has the desired effect)', [])
('good (beneficial; worthwhile, helpful)', [])
('good (of food, edible; not stale or rotten)', [])
('good (useful for a particular purpose)\nAntonyms: huono, kehno\ngood (of high quality)\ngood (competent, skilled, talented) [with inessive or (with verbs) illative of third infinitive ‘at’]\nhyvä uinnissa / hyvä uimaan ― good at swimming\ngood (effective, that has the desired effect)\ngood (beneficial; worthwhile, helpful)\ngood (of food, edible; not stale or rotten)', ['hyvä uinnissa / hyvä uimaan ― good at swimming'])
('good (acting in the interest of good; ethical good intentions)\nAntonym: paha\nKaikilla on taito tehdä hyvää. ― Everyone has the ability to do good (deeds).', ['Kaikilla on taito teh

In [22]:
def extract_definition(definition):
    translation = definition.select_one('a').text
    desc = definition.select_one('.gloss-content').text
    examples = [ex.text for ex in definition.select('.h-usage-example')]

    return (translation, desc, examples)

translations = adj.select('section:has(> h3:-soup-contains("Adjective")) > ol > li')

extract_definition(translations[0].select('ol > li')[1])

('good', 'of high quality', [])